In [2]:
import simpy
import pandas as pd
import random

In [8]:
class ProcessStep:
    def __init__(self, env, name, worker_capacity, processing_time):
        self.env = env
        self.name = name
        self.worker_capacity = worker_capacity
        self.processing_time = processing_time
        self.queue = simpy.Store(env)

    @property
    def process_entity(self):
        while True:
            entity = yield self.queue.get()
            yield self.env.timeout(self.processing_time)
            timestamp = self.env.now
            worker_id = entity.worker_id
            step_number = self.name
            data = {
                'Processed_Entity_ID': entity.id,
                'Worker_ID': worker_id,
                'Step_Number': step_number,
                'Timestamp': timestamp
            }
            print(data)
            results.append(data)

class Entity:
    def __init__(self, id, worker_id):
        self.id = id
        self.worker_id = worker_id

def put_entity(queue, entity):
    yield queue.put(entity)

def entity_generator(env, steps):
    for i in range(1, 11):  
        worker_id = random.randint(1, 5)  
        entity = Entity(i, worker_id)
        yield env.process(put_entity(steps[0].queue, entity))
        yield env.timeout(0.1)
        for step in steps[1:]:
            yield env.process(put_entity(step.queue, entity))
            yield env.timeout(0.1)

env = simpy.Environment()
results = []

entrada = ProcessStep(env, "Entrada Material Sucio", worker_capacity=5, processing_time=1)
cargado = ProcessStep(env, "Cargado en carro L+D", worker_capacity=5, processing_time=3)
initiada = ProcessStep(env, "Carga L+D initiada", worker_capacity=5, processing_time=5)
liberada = ProcessStep(env, "Carga L+D liberada", worker_capacity=5, processing_time=75)
montaje = ProcessStep(env, "Montaje", worker_capacity=5, processing_time=1)
montada = ProcessStep(env, "Producción montada", worker_capacity=5, processing_time=15)
cargas = ProcessStep(env, "Composición de cargas", worker_capacity=5, processing_time=50)
esterilizador = ProcessStep(env, "Carga de esterilizador liberada", worker_capacity=5, processing_time=110)
comisionado = ProcessStep(env, "Comisionado", worker_capacity=5, processing_time=150)

steps = [entrada, cargado, initiada, liberada, montaje, montada, cargas, esterilizador, comisionado]

for step in steps:
    env.process(step.process_entity)

env.process(entity_generator(env, steps))
env.run()

df = pd.DataFrame(results)
df.to_csv('/Users/abdalrhman/Documents/bdma-thesis/python/data/simulation_results.csv', index=False)


{'Processed_Entity_ID': 1, 'Worker_ID': 1, 'Step_Number': 'Entrada Material Sucio', 'Timestamp': 1}
{'Processed_Entity_ID': 1, 'Worker_ID': 1, 'Step_Number': 'Montaje', 'Timestamp': 1.4}
{'Processed_Entity_ID': 2, 'Worker_ID': 1, 'Step_Number': 'Entrada Material Sucio', 'Timestamp': 2}
{'Processed_Entity_ID': 2, 'Worker_ID': 1, 'Step_Number': 'Montaje', 'Timestamp': 2.4}
{'Processed_Entity_ID': 3, 'Worker_ID': 1, 'Step_Number': 'Entrada Material Sucio', 'Timestamp': 3}
{'Processed_Entity_ID': 1, 'Worker_ID': 1, 'Step_Number': 'Cargado en carro L+D', 'Timestamp': 3.1}
{'Processed_Entity_ID': 3, 'Worker_ID': 1, 'Step_Number': 'Montaje', 'Timestamp': 3.4}
{'Processed_Entity_ID': 4, 'Worker_ID': 2, 'Step_Number': 'Entrada Material Sucio', 'Timestamp': 4}
{'Processed_Entity_ID': 4, 'Worker_ID': 2, 'Step_Number': 'Montaje', 'Timestamp': 4.4}
{'Processed_Entity_ID': 5, 'Worker_ID': 1, 'Step_Number': 'Entrada Material Sucio', 'Timestamp': 5}
{'Processed_Entity_ID': 1, 'Worker_ID': 1, 'Step_Num